In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("D:/Professional/Hackules/LLM/source/2_news_research_tool_project/notebooks/sample_text.csv")

In [5]:
df.shape

(8, 2)

### Step 1 : Create source embeddings for the text column

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [9]:
vectors.shape

(8, 768)

In [13]:
dim = vectors.shape[1]

### Step 2 : Build a FAISS Index for vectors

In [14]:
import faiss
index = faiss.IndexFlat(dim)

In [18]:
index

<faiss.swigfaiss.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x0000026219B87B40> >

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [19]:
index.add(vectors)

In [20]:
index

<faiss.swigfaiss.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x0000026219B87B40> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [53]:
search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [54]:
vec

array([ 1.77042987e-02,  7.70549327e-02, -1.25192860e-02, -1.39330281e-03,
        5.92104625e-03, -1.42679466e-02, -4.72583883e-02,  3.91951529e-03,
        4.41600755e-02,  3.09807751e-02,  9.15983319e-02,  2.02603173e-02,
       -2.16004848e-02,  1.33208251e-02, -4.14275751e-02,  4.16395329e-02,
        4.37019058e-02,  6.06421428e-03,  2.21781358e-02, -1.94715448e-02,
       -3.44190598e-02,  3.79219651e-02,  5.27903205e-03,  2.22011227e-02,
       -1.11636510e-02, -2.53351014e-02,  7.79388323e-02,  3.47348116e-02,
        1.20056737e-02, -6.52654245e-02, -4.83940803e-02, -3.03445589e-02,
       -6.99562803e-02, -2.53461506e-02,  1.78119126e-06,  1.50222108e-02,
        2.15191785e-02,  2.21124627e-02, -8.38034824e-02,  3.70961474e-03,
       -2.21118201e-02, -8.53062198e-02,  6.44917018e-04,  1.62597895e-02,
       -1.88165400e-02,  3.99217904e-02, -3.13138999e-02,  2.47449577e-02,
       -7.98757188e-03,  6.11144528e-02, -1.47846481e-02,  4.10300709e-05,
        1.20446468e-02,  

In [55]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [56]:
faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [57]:
new_vec = ""

In [58]:
distance, I = index.search(svec, k=2)

In [59]:
distance

array([[1.3433162, 1.7125276]], dtype=float32)

In [60]:
I

array([[1, 0]], dtype=int64)

In [61]:
I.tolist()

[[1, 0]]

In [62]:
row_indices = I.tolist()[0]
row_indices

[1, 0]

In [63]:
df.loc[row_indices]

,text,category
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
0,Meditation and yoga can improve mental health,Health
